# Movie Recommendation Engine Tutorial

This is a Notebook adaptation of a Movielens-based example on Codementor.

* [Part 1](https://www.codementor.io/@jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw)
* [Part 2](https://www.codementor.io/@jadianes/building-a-web-service-with-apache-spark-flask-example-app-part2-du1083854)

Before running this notebook, make sure that you have run `setup.sh` in the repo root to download movie recommendation data files.

## Setup

Import Python modules that will be used for building and training the recommendation model

In [1]:
import os
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import *

Configure Pandas (used for display/debug purposes)

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

Work out the paths to the files and directories that are relevant to the model build

In [3]:
small_ratings_file = os.path.join('/data', 'ml-latest-small', 'ratings.csv')
complete_ratings_file = os.path.join('/data', 'ml-latest', 'ratings.csv')
complete_movies_file = os.path.join('/data', 'ml-latest', 'movies.csv')

model_path = os.path.join('/model', 'movie_lens_als')

Set up the Spark session

In [4]:
spark = SparkSession.builder \
    .appName("Recommender") \
    .master("local[*]") \
    .config("spark.driver.memory", "4G") \
    .config('spark.driver.memory', '16G') \
    .config('spark.driver.maxResultSize', '10G') \
    .getOrCreate()

## Build & Train on Small Dataset

This section loads a small dataset in order to train the model, i.e. tweak parameters to yield the best ALS result. The result of this step will be used in the next stage, using a larger dataset, to retrain the model for release.

In [5]:
small_ratings_raw_data = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load(small_ratings_file)
  
small_ratings_raw_data.createOrReplaceTempView("ratings")

small_ratings_raw_data.limit(5).toPandas()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Transform the raw rating dataset to something that can be used during model training

In [6]:
small_ratings_data = spark.sql("select userId, movieId, rating from ratings")

Split the dataset into three subsets: training, validation, and test. These will be used in training the model below.

In [7]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6.0, 2.0, 2.0], seed=0)

Train the ALS recommendation model by testing out three different ranks.

For each rank, run an ALS model to determine which is most accurate. The best rank will be carried into building the model to use in production.

In [8]:
seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    als = ALS(maxIter=iterations, regParam=regularization_parameter, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", rank=rank)
    model = als.fit(training_RDD)

    # Evaluate the model by computing the RMSE on the test data
    predictions = model.transform(test_RDD)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
    error = evaluator.evaluate(predictions)
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.9102254780971206
For rank 8 the RMSE is 0.9104749136764886
For rank 12 the RMSE is 0.9120549952009228
The best model was trained with rank 4


# Build the Production Model on a Larger Dataset

Load and prepare the dataset

In [9]:
complete_ratings_raw_data = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load(complete_ratings_file)
  
complete_ratings_raw_data.createOrReplaceTempView("ratings")

complete_ratings_data = spark.sql("select userId, movieId, rating from ratings").cache()

print('There are %s recommendations in the complete dataset' % (complete_ratings_data.count()))


There are 27753444 recommendations in the complete dataset


Using the parameters determined during model training on the smaller recommendation dataset, retrain based on a larger volume of data to produce a version of the model that is production ready

In [10]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7.0, 3.0], seed=0)

als = ALS(maxIter=iterations, regParam=regularization_parameter, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", rank=best_rank)
complete_model = als.fit(training_RDD)

Create a pipeline and get the model ready for export. Model pipelines are used by the Arc framework, so the recommendation model needs to be compliant.

In [11]:
pipeline = Pipeline(stages=[complete_model])
model = pipeline.fit(training_RDD)

Export the model ready for use

In [12]:
model.write() \
  .overwrite() \
  .save(model_path)